<a href="https://colab.research.google.com/github/navrat/semi_sup_learning_text_clf/blob/main/Semi_Supervised_self_training_LOGIT_and_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle

In [2]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.21.6
1.0.2


In [7]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# !pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

In [10]:
# import numpy as np
# np.__version__

In [11]:
# %cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
# !ls

In [12]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [13]:
# Sample dataset run 
data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()


11314 documents
20 categories



In [14]:
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
type(y_train)

numpy.ndarray

In [ ]:
# Sample SGD - Log Loss(Logistic regression).
if __name__ == "__main__":
  # Parameters
  sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
  vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

    n_list = [10, 20, 30, 40, 50]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))
      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")
      
      eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), threshold = t, verbose=True)),
        ])
        eval_and_print_metrics(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.709
Accuracy Score:  0.7090844821491693
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 2569 new labels.
End of iteration 2, added 761 new labels.
End of iteration 3, added 211 new labels.
End of iteration 4, added 69 new labels.
End of iteration 5, added 35 new labels.
End of iteration 6, added 17 new labels.
End of iteration 7, added 14 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 6 new labels.
End of iteration 10, added 5 new labels.
Micro-averaged F1 score on test set

In [15]:
# Creating the evaluation functiont to help generate metrics for both supervised and semi-supervised runs
def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1




### Starting with NEWSGROUP dataset

In [38]:
data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

11314 documents
20 categories



In [40]:
data.data[0:5]
# Data column consists of the actual news article with headline as subject and body with a few extra identifiers that we ignore for now

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [43]:
print(data.target)
import collections
collections.Counter(data.target)
# Overall 20 labels i.e. 20 types of news classifications which look fairly balanced except in some cases

[7 4 4 ... 3 1 8]


Counter({0: 480,
         1: 584,
         2: 591,
         3: 590,
         4: 578,
         5: 593,
         6: 585,
         7: 594,
         8: 598,
         9: 597,
         10: 600,
         11: 595,
         12: 591,
         13: 594,
         14: 593,
         15: 599,
         16: 546,
         17: 564,
         18: 465,
         19: 377})

#### Running individual sklearn algorithms

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))
      print("Supervised MNB on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_ng = df_nb_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int(len(X_u50)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = None, kbest = kbest)
        df_nb_ng = df_nb_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.545
Accuracy Score:  0.545068928950159
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  1076 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 1076 new labels.
End of iteration 2, added 1076 new labels.
End of iteration 3, added 1076 new labels.
End of iteration 4, added 1076 new labels.
End of iteration 5, added 3 new labels.
Micro-averaged F1 score on test set: 0.055
Accuracy Score:  0.055496641922940966
----------

---------------------------------K-Best =  861 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training s

In [ ]:
display(df_nb_ng[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,869.0,0.0,NaN,NaN,0.545069
1,869.0,4307.0,NaN,1076.0,0.055497
2,869.0,4307.0,NaN,861.0,0.055143
3,869.0,4307.0,NaN,717.0,0.054436
4,869.0,4307.0,NaN,615.0,0.054083
5,869.0,4307.0,NaN,538.0,0.053376
6,1695.0,0.0,NaN,NaN,0.706257
7,1695.0,4307.0,NaN,1076.0,0.063273
8,1695.0,4307.0,NaN,861.0,0.061152
9,1695.0,4307.0,NaN,717.0,0.060445


In [ ]:
df_nb_ng.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best               NaN       538.0     615.0     717.0     861.0     1076.0
Labeled UnLabeled                                                            
869.0   0.0        0.545069       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.053376  0.054083  0.054436  0.055143  0.055497
1695.0  0.0        0.706257       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.059738  0.059738  0.060445  0.061152  0.063273
2541.0  0.0        0.742312       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.106045  0.073524  0.074585  0.072464  0.079533
3349.0  0.0        0.770944       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.074231  0.078119  0.082715  0.095794  0.111700
4178.0  0.0        0.799222       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.094026  0.099328  0.105338  0.116649  0.142807

In [ ]:
# Random Forest for NG
from sklearn.ensemble import RandomForestClassifier

# Parameters
# mnb_params = 
rf_params = dict(n_estimators=100, random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_rf_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', RandomForestClassifier(**rf_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_rf_ng = df_rf_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(RandomForestClassifier(**rf_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_rf_ng = df_rf_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.654
Accuracy Score:  0.6535878402262284
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 634 new labels.
End of iteration 2, added 421 new labels.
End of iteration 3, added 476 new labels.
End of iteration 4, added 724 new labels.
End of iteration 5, added 842 new labels.
End of iteration 6, added 877 new labels.
End of iteration 7, added 266 new labels.
End of iteration 8, added 57 new labels.
End of iteration 9, added 10 new labels.
Micro-averaged F1 score on test set: 0.198
Accuracy Score:  0.1975963237

In [ ]:
df_rf_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.653588       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.197596  0.123719  0.253800  0.574054  0.645811
1695.0  0.0        0.725345       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.275362  0.341110  0.654295  0.705196  0.726405
2541.0  0.0        0.758925       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.578296  0.674797  0.722517  0.738423  0.747967
3349.0  0.0        0.783669       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.728526  0.734182  0.744079  0.765642  0.780134
4178.0  0.0        0.791092       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.765288  0.772711  0.770944  0.790032  0.793213

In [ ]:
## SGD (Logistic Regression) experiment

In [ ]:
# Logistic Regression
from sklearn.linear_model import SGDClassifier

# Parameters
sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

if __name__ == "__main__":

    df_sgd_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**sgd_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_sgd_ng = df_sgd_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_sgd_ng = df_sgd_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.709
Accuracy Score:  0.7090844821491693
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 2569 new labels.
End of iteration 2, added 761 new labels.
End of iteration 3, added 211 new labels.
End of iteration 4, added 69 new labels.
End of iteration 5, added 35 new labels.
End of iteration 6, added 17 new labels.
End of iteration 7, added 14 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 6 new labels.
End of iteration 10, added 5 new labels.
Micro-averaged F1 score on test set

In [ ]:
df_sgd_ng

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,869.0,0.0,NaN,NaN,0.709084
1,869.0,4307.0,0.4,NaN,0.795688
2,869.0,4307.0,0.5,NaN,0.793920
3,869.0,4307.0,0.6,NaN,0.795688
4,869.0,4307.0,0.7,NaN,0.782609
5,869.0,4307.0,0.8,NaN,0.763874
6,1695.0,0.0,NaN,NaN,0.806999
7,1695.0,4307.0,0.4,NaN,0.848003
8,1695.0,4307.0,0.5,NaN,0.853659
9,1695.0,4307.0,0.6,NaN,0.849417


In [ ]:
df_sgd_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.709084       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.795688  0.793920  0.795688  0.782609  0.763874
1695.0  0.0        0.806999       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.848003  0.853659  0.849417  0.847296  0.835277
2541.0  0.0        0.836338       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.868858  0.869565  0.867798  0.860021  0.856486
3349.0  0.0        0.858254       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.885118  0.882998  0.879463  0.872747  0.868505
4178.0  0.0        0.869919       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.895369  0.896430  0.889360  0.887593  0.882998

Clear improvement in accuracies (pivot above) and micro-F1 (console output) can be seen upon using self training vs. training only on labeled dataset

In [ ]:
## MLP Classifier 

In [ ]:
# MLP for NG
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,), max_iter=100,activation = 'relu',solver='lbfgs',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive') # regularization is by default based on alpha =0.0001
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised MLPClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline      
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('scale', StandardScaler(with_mean=False)),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('scale', StandardScaler(with_mean=False)),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.516
Accuracy Score:  0.5164369034994698
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 1676 new labels.
End of iteration 2, added 1256 new labels.
End of iteration 3, added 556 new labels.
End of iteration 4, added 365 new labels.
End of iteration 5, added 127 new labels.
End of iteration 6, added 75 new labels.
End of iteration 7, added 30 new labels.
End of iteration 8, added 21 new labels.
End of iteration 9, added 7 new labels.
End of iteration 10, added 33 new labels.
Micro-averaged F1 score on tes

In [ ]:
df_mlp_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']


Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.516437       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.560622  0.491340  0.503005  0.527041  0.507600
1695.0  0.0        0.645458       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.682220  0.685048  0.684341  0.664546  0.659244
2541.0  0.0        0.715447       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.717568  0.737363  0.755744  0.723224  0.707671
3349.0  0.0        0.747614       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.767762  0.767055  0.767409  0.762814  0.769883
4178.0  0.0        0.761753       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.788971  0.802050  0.792506  0.784730  0.785790

**Even** for an MLP, the same situation holds true. Micro-F1 and Accuracy show improvements using selftrainingclassifier and usually the higher tresholds result in better performance.

#### End of algorithm-runs based off of self training classifiers on NEWSGROUP dataset.

#### Starting algorithm runs on IMDB dataset

In [46]:
# Creating a simple train_test validation split and clean the dataset against punctuations, HTML tags, and URLs
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = unlabeled_per_class   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs



import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data



def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    # train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    # lemmatizer = WordNetLemmatizer()
    # train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Starting by reading the IMDB dataset and visualizing the top 5 rows

In [47]:
data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
train_df = pd.read_csv(data_path+'train.csv', header=None)
test_df = pd.read_csv(data_path+'test.csv', header=None)
print("Read data successful", train_df.shape, test_df.shape)
print(train_df.head(5))
test_df.head()
# Two columns per dataset - review and sentiment (1 is a positive sentiment and 0 is a negative sentiment)

Read data successful (37500, 2) (12500, 2)
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1


,0,1
0,I really liked this Summerslam due to the look...,1
1,Not many television shows appeal to quite as m...,1
2,The film quickly gets to a major chase scene w...,0
3,Jane Austen would definitely approve of this o...,1
4,Expectations were somewhat high for me when I ...,0


In [48]:
# Sample Run to see if everything runs fine
import os

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report



# Parameters
sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sgd_params)),
])

# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), verbose=True)),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # if -1 in y_train:
    #   print("Y-PRED-PROBA", clf.predict_proba(X_train))

    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("Classification Report: \n", classification_report(y_test, y_pred))
    print("-" * 10)
    print()



def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [50]:
# Sample Run to see if everything runs fine

if __name__ == "__main__":

    # n_list = [10, 200, 500]
    n_list = [10, 200]
    threshold=[0.4, 0.7]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised SGDClassifier on ", 2*n," of the training data:")
      eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # y_train[~y_mask] = -1
      print("SelfTrainingClassifier on ", 2*n," of the training data (rest "
          "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), threshold = t, verbose=True)),
        ])

        eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on  20  of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.565
Accuracy Score:  0.56504
Classification Report: 
               precision    recall  f1-score   support

           0       0.55      0.68      0.61      6157
           1       0.59      0.45      0.51      6343

    accuracy                           0.57     12500
   macro avg       0.57      0.57      0.56     12500
weighted avg       0.57      0.57      0.56     12500

----------

SelfTrainingClassifier on  20  of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.508
Accuracy Score:  0.50816
Classifi

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Supervised SGDClassifier on  400  of the training data:
Number of training samples: 400
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.772
Accuracy Score:  0.77224
Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.76      0.77      6157
           1       0.77      0.79      0.78      6343

    accuracy                           0.77     12500
   macro avg       0.77      0.77      0.77     12500
weighted avg       0.77      0.77      0.77     12500

----------

SelfTrainingClassifier on  400  of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5400
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.805
Accuracy Score:  0.80544
Classification Report: 
               precision    recall  f1-score   support

           0       0.

Everything looks fine. Continuing with individual algoirithms

### Adding an additional layer to the experiment- Considering 2 volumes of unlabeled data
- 2000 (1000 observations per class passed as unlabeled)
- 5000 (2500 observations per class passed as unlabeled)


In [51]:
# Naive Bayes for IMDB with 2k unlabeled observations
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_imdb_2k = df_nb_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int((unlabeled_per_class*2)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = kbest)
        df_nb_imdb_2k = df_nb_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.590
Accuracy Score:  0.59016
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  500 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 500 new labels.
End of iteration 2, added 500 new labels.
End of iteration 3, added 500 new labels.
End of iteration 4, added 500 new labels.
Micro-averaged F1 score on test set: 0.493
Accuracy Score:  0.49256
----------

---------------------------------K-Best =  400 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 400 new labels.
End of iteration 2, added 400 new

In [52]:
display(df_nb_imdb_2k[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,20.0,0.0,NaN,NaN,0.59016
1,20.0,2000.0,NaN,500.0,0.49256
2,20.0,2000.0,NaN,400.0,0.49256
3,20.0,2000.0,NaN,333.0,0.49256
4,20.0,2000.0,NaN,285.0,0.49256
5,20.0,2000.0,NaN,250.0,0.49256
6,100.0,0.0,NaN,NaN,0.68456
7,100.0,2000.0,NaN,500.0,0.49256
8,100.0,2000.0,NaN,400.0,0.49256
9,100.0,2000.0,NaN,333.0,0.49256


In [53]:
df_nb_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best               NaN      250.0    285.0    333.0    400.0    500.0
Labeled UnLabeled                                                      
20.0    0.0        0.59016      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
100.0   0.0        0.68456      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
400.0   0.0        0.79504      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
1000.0  0.0        0.83752      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
2000.0  0.0        0.85776      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49280  0.49504

No improvements can be seen on IMDB using Naive Bayes. Self Training reduces the supervised accuracies


In [54]:
# Naive Bayes for IMDB with 5k unlabeled observations
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_imdb_5k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500
      
      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_imdb_5k = df_nb_imdb_5k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int((unlabeled_per_class*2)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = kbest)
        df_nb_imdb_5k = df_nb_imdb_5k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.590
Accuracy Score:  0.59016
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  1250 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 1250 new labels.
End of iteration 2, added 1250 new labels.
End of iteration 3, added 1250 new labels.
End of iteration 4, added 1250 new labels.
Micro-averaged F1 score on test set: 0.493
Accuracy Score:  0.49256
----------

---------------------------------K-Best =  1000 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 1000 new labels.
End of iteration 2, added 

In [55]:
display(df_nb_imdb_5k[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,20.0,0.0,NaN,NaN,0.59016
1,20.0,5000.0,NaN,1250.0,0.49256
2,20.0,5000.0,NaN,1000.0,0.49256
3,20.0,5000.0,NaN,833.0,0.49256
4,20.0,5000.0,NaN,714.0,0.49256
5,20.0,5000.0,NaN,625.0,0.49256
6,100.0,0.0,NaN,NaN,0.68456
7,100.0,5000.0,NaN,1250.0,0.49256
8,100.0,5000.0,NaN,1000.0,0.49256
9,100.0,5000.0,NaN,833.0,0.49256


In [56]:
df_nb_imdb_5k.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best              NaN      625.0    714.0    833.0    1000.0   1250.0
Labeled UnLabeled                                                      
20.0    0.0        0.59016      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
100.0   0.0        0.68456      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
400.0   0.0        0.79504      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
1000.0  0.0        0.83752      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
2000.0  0.0        0.85776      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256

Similar degradation is seen at higher volumes of unlabeled data as well



### Running Logistic Regression next on 2k and 5k unlabeled observations

In [59]:
# Logistic Regression using SGD (log-Loss) for IMDB  with 2k unlabeled observations
from sklearn.linear_model import SGDClassifier

# Parameters
sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

if __name__ == "__main__":

    df_sgd_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**sgd_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_sgd_imdb_2k = df_sgd_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_sgd_imdb_2k = df_sgd_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.565
Accuracy Score:  0.56504
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.515
Accuracy Score:  0.51472
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.515
Accuracy Score:  0.51472
----------

---------------------------------Threshold =  0.6 --------------------

In [60]:
df_sgd_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.56504      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.51472  0.51472  0.49280  0.49256  0.49256
100.0   0.0        0.67512      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.70520  0.70520  0.70752  0.68592  0.60728
400.0   0.0        0.77224      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.79688  0.79688  0.79968  0.79776  0.79872
1000.0  0.0        0.83112      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.83976  0.83976  0.83848  0.84192  0.83792
2000.0  0.0        0.85648      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.86304  0.86304  0.86136  0.86088  0.86104

At very low volumes, logistic regression does not work BUT at higher volumes of labeled data from 100 onward, there is significant lift through semi-supervised learning

In [61]:
# Logistic Regression using SGD (log-Loss) for IMDB  with 5k unlabeled observations
from sklearn.linear_model import SGDClassifier

# Parameters
sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_sgd_imdb_5k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**sgd_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_sgd_imdb_5k = df_sgd_imdb_5k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_sgd_imdb_5k = df_sgd_imdb_5k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.565
Accuracy Score:  0.56504
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.508
Accuracy Score:  0.50816
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.508
Accuracy Score:  0.50816
----------

---------------------------------Threshold =  0.6 --------------------

In [62]:
df_sgd_imdb_5k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.56504      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.50816  0.50816  0.49264  0.49256  0.49256
100.0   0.0        0.67512      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.71096  0.71096  0.71696  0.70568  0.61736
400.0   0.0        0.77224      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.80544  0.80544  0.80352  0.80784  0.80520
1000.0  0.0        0.83112      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.84456  0.84456  0.84144  0.84072  0.83704
2000.0  0.0        0.85648      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.86568  0.86568  0.86440  0.86464  0.86280

Similar pattern as 2k unlabeled volumes is observed with 5k unlabeled volumes.
In all cases, the higher unlabeled volume (5k) has generated a stronger lift than at lower volume (2k) 

### Running MLP next on 2k and 5k unlabeled observations

In [63]:
# MLP for IMDB at 2k unlabeled volume
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,), max_iter=100,activation = 'relu',solver='lbfgs',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive') # regularization is by default based on alpha =0.0001
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('scale', StandardScaler(with_mean=False)),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('scale', StandardScaler(with_mean=False)),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.605
Accuracy Score:  0.60472
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.576
Accuracy Score:  0.57592
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.576
Accuracy Score:  0.57592
----------

---------------------------------Threshold =  0.6 --------------------

In [64]:
df_mlp_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.60472      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.57592  0.57592  0.55792  0.52288  0.49520
100.0   0.0        0.66304      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.61320  0.61320  0.62944  0.65256  0.65184
400.0   0.0        0.75648      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.73272  0.73272  0.74896  0.75168  0.76048
1000.0  0.0        0.81464      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.78672  0.78672  0.78440  0.78536  0.78640
2000.0  0.0        0.81368      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.82472  0.82472  0.82552  0.83336  0.83248

In [65]:
# MLP for IMDB at 5k unlabeled volume
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,), max_iter=100,activation = 'relu',solver='lbfgs',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive') # regularization is by default based on alpha =0.0001
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_5k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('scale', StandardScaler(with_mean=False)),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_5k = df_mlp_imdb_5k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('scale', StandardScaler(with_mean=False)),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_5k = df_mlp_imdb_5k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.605
Accuracy Score:  0.60472
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.578
Accuracy Score:  0.5784
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 5000 new labels.
Micro-averaged F1 score on test set: 0.578
Accuracy Score:  0.5784
----------

---------------------------------Threshold =  0.6 ----------------------

KeyboardInterrupt: ignored

In [66]:
df_mlp_imdb_5k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.60472      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.57840  0.57840  0.60032  0.59192  0.55056
100.0   0.0        0.66304      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.61040  0.61040  0.63040  0.66008  0.67576
400.0   0.0        0.75648      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.66896  0.66896  0.65712  0.60000  0.52728
1000.0  0.0        0.81464      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.77800  0.77800  0.79040  0.79320  0.79040
2000.0  0.0        0.81368      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.81728      NaN      NaN      NaN      NaN

5k does not bring any difference. The results are not great with a shallow netowrk. Attempting it with a denser network and a better optimizer

In [72]:
# MLP config2 for IMDB at 2k unlabeled volume
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('scale', StandardScaler(with_mean=False)),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('scale', StandardScaler(with_mean=False)),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.575
Accuracy Score:  0.57512
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.498
Accuracy Score:  0.49848
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.498
Accuracy Score:  0.49848
----------

---------------------------------Threshold =  0.6 --------------------

In [73]:
df_mlp_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.57512      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49848  0.49848  0.50240  0.50192  0.49672
100.0   0.0        0.68056      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.51424  0.51424  0.50768  0.51216  0.52224
400.0   0.0        0.76896      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.74936  0.74936  0.74664  0.75320  0.77576
1000.0  0.0        0.80320      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.78872  0.78872  0.78472  0.78688  0.80048
2000.0  0.0        0.81096      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.80984  0.80984  0.81280  0.80136  0.81528

Using the keras approach to neural network to see if things get better

In [75]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier

tf.random.set_seed(0)
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(100, activation='relu'),
      tf.keras.layers.Dense(50),
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                # loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model



model = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


In [78]:
# MLP config2 using Keras for IMDB at 2k unlabeled volume
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
      ('vect', CountVectorizer(**vectorizer_params)),
      ('tfidf', TfidfTransformer()), 
      # ('kbest', MySelectKBest(f_classif, k=20000)),         
      ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
      ('clf', model)
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
        tf.keras.backend.clear_session()  

        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            # ('kbest', MySelectKBest(f_classif, k=20000)),
            ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', SelfTrainingClassifier(model, criterion = 'threshold', threshold = t, verbose=True)),
        ])

        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.514
Accuracy Score:  0.51448
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 650 new labels.
End of iteration 2, added 1350 new labels.
Micro-averaged F1 score on test set: 0.673
Accuracy Score:  0.67256
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 855 new labels.
End of iteration 2, added 1145 new labels.
Micro-averaged F1 score on test set: 0.692
Accuracy Score:  0.6917

In [79]:
df_mlp_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.51448      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.67256  0.69176  0.49360  0.49256  0.49256
100.0   0.0        0.66328      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.72352  0.69264  0.71200  0.73744  0.73368
400.0   0.0        0.79488      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.81424  0.81392  0.81432  0.81480  0.81744
1000.0  0.0        0.84008      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.85144  0.84888  0.85200  0.85208  0.84904
2000.0  0.0        0.85952      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.86824  0.86904  0.86888  0.86856  0.86912

Significant boost can be observed at various labeled volume levels. Trying this at 5k unlabeled volume to see if it improves

In [83]:
# MLP config2 using Keras for IMDB at 5k unlabeled volume

model = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_5k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
      ('vect', CountVectorizer(**vectorizer_params)),
      ('tfidf', TfidfTransformer()), 
      # ('kbest', MySelectKBest(f_classif, k=20000)),         
      ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
      ('clf', model)
      ])
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_5k = df_mlp_imdb_5k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
        tf.keras.backend.clear_session()  

        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            # ('kbest', MySelectKBest(f_classif, k=20000)),
            ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', SelfTrainingClassifier(model, criterion = 'threshold', threshold = t, verbose=True)),
        ])

        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_5k = df_mlp_imdb_5k.append(temp, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.537
Accuracy Score:  0.53704
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 2759 new labels.
End of iteration 2, added 2241 new labels.
Micro-averaged F1 score on test set: 0.507
Accuracy Score:  0.50744
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 1162 new labels.
End of iteration 2, added 3838 new labels.
Micro-averaged F1 score on test set: 0.544
Accuracy Score:  0.54

In [84]:
df_mlp_imdb_5k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.53704      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.50744  0.54416  0.49256  0.50744  0.49256
100.0   0.0        0.67728      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.75280  0.70328  0.75552  0.75608  0.62552
400.0   0.0        0.79272      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.82752  0.82760  0.82600  0.82600  0.82432
1000.0  0.0        0.83960      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.84776  0.85120  0.84952  0.84960  0.85384
2000.0  0.0        0.85944      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.87024  0.86888  0.87152  0.87264  0.87008

### There is some additional benefit in most cases in using additional unlabeled volume

In [85]:
#Checking if threshold parameter for SelfTrainingClassifier is used for adding examples selectively or as a threshold to predict_proba.
#If it is used for predict_proba, then we should compare it with the supervised version with the same threshold - not with supervised(threshold=0.5).

def eval_and_print_metrics_2(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("-" * 10)
    print()
    return y_pred, y_pred_proba



import pandas as pd
if __name__ == "__main__":

    # n_list = [10, 200, 500]
    n_list = [10, 50, 200, 500, 1000]
    n_list = [200]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]
    threshold = [0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './drive/MyDrive/semi_sup_learning/data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      # X_20, y_20 = texts[:2*n], labels[:2*n]
      # print("Supervised SGDClassifier on ", 2*n," of the training data:")
      # eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # y_train[~y_mask] = -1
      print("SelfTrainingClassifier on ", 2*n," of the training data (rest "
          "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), threshold = t, verbose=True)),
        ])

        y_pred, y_proba = eval_and_print_metrics_2(st_pipeline, X_train, y_train, X_test, y_test)

        print("t = ",t, '\n', y_pred)
        print("t = ",t, '\n', y_proba)

        print("Number of equal values between y_pred and y_pred(calculated) when we calculate y_pred from y_proba with threshold = 0.8: ", sum(np.equal((y_proba[:,1] >= 0.8).astype(int), y_pred)))
        print("Number of equal values between y_pred and y_pred(calculated) when we calculate y_pred from y_proba with threshold = 0.5: ", sum(np.equal((y_proba[:,1] >= 0.5).astype(int), y_pred)))


#Conclusion: Threshold is used as a part of training but not to calculate pred from pred_proba.



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 400~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SelfTrainingClassifier on  400  of the training data (rest is unlabeled):
---------------------------------Threshold =  0.8 ---------------------------------
Number of training samples: 5400
Unlabeled samples in training set: 5000
End of iteration 1, added 2376 new labels.
End of iteration 2, added 1084 new labels.
End of iteration 3, added 280 new labels.
End of iteration 4, added 95 new labels.
End of iteration 5, added 45 new labels.
End of iteration 6, added 23 new labels.
End of iteration 7, added 8 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 9 new labels.
End of iteration 10, added 3 new labels.
Micro-averaged F1 score on test set: 0.805
Accuracy Score:  0.8052
----------

t =  0.8 
 [0 0 0 ... 0 1 1]
t =  0.8 
 [[0.89715549 0.10284451]
 [0.57326692 0.42673308]
 [0.93212653 0.06787347]
 ...
 [0.74134373 0.25865627]
 [0.00151803 0.99848197]
 [0.0892771